# 📝 License and Notices

This notebook is part of "Arduino 2D Pan-Tilt Control System + AI Mosquito Auto-Tracking" (mosquito-pt2d).

Copyright 2025 Arduino PT2D Project

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this work except in compliance with the License. You may obtain a copy of the License at:

- http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

For project and third-party notices, see the repository files:
- LICENSE
- NOTICE

## 1. 掛載 Google Drive

## 📋 完整工作流程總覽

### 階段 1: Colab 訓練 (本 Notebook)

| 步驟 | Cell | 輸出 | 說明 |
|------|------|------|------|
| 1. 掛載 Drive | Cell 2 | - | Google Drive 連結 |
| 2. 安裝環境 | Cell 3 | - | ultralytics, torch |
| 3. 準備數據集 | Cell 4 | `training_dataset/` | 80/20 分割 |
| 4. 訓練模型 | Cell 5 | `runs/detect/mosquito_dual_platform/` | 100 epochs |
| 5. 評估結果 | Cell 6 | mAP50, mAP50-95 | 精度指標 |
| 6. 保存模型 | Cell 7 | `models/mosquito_yolov8_new.pt` | ⚠️ 關鍵檔案 |
| 7. 路徑驗證 | Cell 8 | - | 確認無斷點 |
| 8. 查看曲線 | Cell 9 | 訓練圖表 | 可視化 |

### 階段 2: 本機部署 (deploy_model.py)

```bash
# 前置條件: Google Drive 已同步
cd d:/Workspaces/mosquito-pt2d
python python/deploy_model.py
```

| 步驟 | 動作 | 輸出 | 說明 |
|------|------|------|------|
| 1. 備份 | 自動 | `mosquito_yolov8_backup_*.pt` | 保護舊模型 |
| 2. 複製 | 自動 | `models/mosquito_yolov8.pt` | 從 Google Drive |
| 3. ONNX | 自動 | `models/mosquito_yolov8.onnx` | 兩平台共用 |
| 4. RKNN | 自動 | `models/mosquito_yolov8.rknn` | Orange Pi 5 |
| 5. RDK X5 | 手動 | `mosquito_yolov8.bin` | 需 hb_mapper |

### 階段 3: 平台部署

**Orange Pi 5** (自動完成):
```bash
# 直接使用 .rknn 模型
python python/streaming_tracking_system.py
```

**RDK X5** (需額外步驟):
```bash
# 1. x86 機器編譯 .bin
python3 mapper.py --onnx models/mosquito_yolov8.onnx --cal-images calibration_images

# 2. 複製到板端並更新 config.py
scp mosquito_yolov8.bin rdkx5:/path/to/models/

# 3. 運行追蹤系統
python python/streaming_tracking_system.py
```

### 🔗 關鍵斷點檢查

✅ **Colab → Google Drive**
- 輸出路徑: `/content/drive/MyDrive/Colab Notebooks/mosquito-training/models/mosquito_yolov8_new.pt`

✅ **Google Drive → 本機**
- 自動同步到: `G:\我的雲端硬碟\Colab Notebooks\mosquito-training\models\`
- deploy_model.py 自動推導路徑（基於 config.RELOCATION_BASE_DIR）

✅ **本機 → 目標板**
- Orange Pi 5: 直接使用 `models/mosquito_yolov8.rknn`
- RDK X5: 手動複製 `mosquito_yolov8.bin`

### 🚀 一鍵命令

```bash
# 完整流程（Orange Pi 5）
python python/deploy_model.py && python python/streaming_tracking_system.py

# 僅更新模型
python python/deploy_model.py

# 跳過 RKNN（僅 ONNX）
python python/deploy_model.py --skip-rknn
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 設定工作目錄（與本機搬遷/部署預期一致）
import os
os.makedirs('/content/drive/MyDrive/Colab Notebooks/mosquito-training', exist_ok=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/mosquito-training')

print("✓ Google Drive 已掛載")
print(f"✓ 工作目錄: {os.getcwd()}")

## 2. 安裝訓練環境

In [ ]:
!pip install ultralytics -q

# 檢查 GPU
import torch
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 名稱: {torch.cuda.get_device_name(0)}")
    print(f"GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ 警告：未偵測到 GPU，請檢查執行階段設定")

## 3. 準備訓練數據集

In [ ]:
import os
import shutil
from pathlib import Path
import random

# 建立目錄結構
os.makedirs('training_dataset/images/train', exist_ok=True)
os.makedirs('training_dataset/images/val', exist_ok=True)
os.makedirs('training_dataset/labels/train', exist_ok=True)
os.makedirs('training_dataset/labels/val', exist_ok=True)

# 選擇樣本來源：優先使用 relocated/<latest>/mosquito，其次 fallback 到 sample_collection/confirmed/mosquito
relocated_base = Path('relocated')
mosquito_dir: Path
if relocated_base.exists():
    subdirs = [d for d in relocated_base.iterdir() if d.is_dir()]
    if subdirs:
        latest = sorted(subdirs)[-1]  # 目錄名稱為時間戳，字串排序即可
        mosquito_dir = latest / 'mosquito'
        print(f"✓ 使用搬遷資料夾樣本: {mosquito_dir}")
    else:
        mosquito_dir = Path('sample_collection/confirmed/mosquito')
        print(f"⚠️ 找不到 relocated/* 子目錄，改用: {mosquito_dir}")
else:
    mosquito_dir = Path('sample_collection/confirmed/mosquito')
    print(f"⚠️ 找不到 relocated 目錄，改用: {mosquito_dir}")

mosquito_images = list(mosquito_dir.glob('*.jpg'))

if len(mosquito_images) == 0:
    print("❌ 錯誤：找不到蚊子樣本圖片")
    print(f"請確認 {mosquito_dir} 目錄存在且包含圖片")
else:
    # 80/20 分割
    random.shuffle(mosquito_images)
    split_idx = int(len(mosquito_images) * 0.8)
    train_images = mosquito_images[:split_idx]
    val_images = mosquito_images[split_idx:]

    # 複製圖片並生成標籤
    for img_list, split in [(train_images, 'train'), (val_images, 'val')]:
        for img_path in img_list:
            # 複製圖片
            shutil.copy(img_path, f'training_dataset/images/{split}/')

            # 生成 YOLO 標籤（整張圖為蚊子）
            label_path = f'training_dataset/labels/{split}/{img_path.stem}.txt'
            with open(label_path, 'w') as f:
                f.write('0 0.5 0.5 1.0 1.0\n')  # class_id x_center y_center width height

    print(f"✓ 訓練集: {len(train_images)} 張")
    print(f"✓ 驗證集: {len(val_images)} 張")

    # 建立 dataset.yaml（使用當前工作目錄的絕對路徑）
    yaml_content = """path: /content/drive/MyDrive/Colab Notebooks/mosquito-training/training_dataset
train: images/train
val: images/val
names:
  0: mosquito
nc: 1
"""

    with open('training_dataset/dataset.yaml', 'w') as f:
        f.write(yaml_content)

    print("✓ 數據集準備完成")

## 4. 開始訓練模型

In [ ]:
from ultralytics import YOLO

# 載入預訓練模型
# 建議：使用 YOLOv8n（nano）以獲得最佳性能/精度平衡
model = YOLO('yolov8n.pt')  # 或使用已訓練的 'models/mosquito_yolov8.pt'

# 訓練參數（適配 RDK X5 與 Orange Pi 5）
# RDK X5 參考: https://github.com/D-Robotics/rdk_model_zoo
# Orange Pi 5 參考: https://github.com/airockchip/rknn-toolkit2
results = model.train(
    data='training_dataset/dataset.yaml',
    epochs=100,            # 充足訓練以提升量化後精度
    imgsz=640,             # 兩平台標準輸入尺寸（YOLOv8n: 640×640）
    batch=16,              # 根據 GPU 記憶體調整
    lr0=0.01,              # 學習率（YOLOv8 預設）
    name='mosquito_dual_platform',
    patience=20,           # 增加 patience 避免過早停止
    device=0,              # 使用 GPU
    workers=4,             # 資料載入執行緒
    amp=True,              # 自動混合精度訓練（加速）
    # 量化友好設定
    close_mosaic=10,       # 最後 10 個 epoch 關閉 Mosaic 增強
    optimizer='AdamW',     # 優化器
    cos_lr=True            # 餘弦學習率衰減
)

print("\n" + "="*50)
print("✓ 訓練完成!")
print(f"✓ 最佳模型: runs/detect/mosquito_dual_platform/weights/best.pt")
print("="*50)

## 5. 評估訓練結果

In [ ]:
# 驗證模型
metrics = model.val()

print("\n訓練指標：")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"精確率: {metrics.box.mp:.4f}")
print(f"召回率: {metrics.box.mr:.4f}")

## 6. 複製模型到 Google Drive

In [ ]:
import shutil
from datetime import datetime

# 複製最佳模型（PyTorch .pt 格式）
best_pt = 'runs/detect/mosquito_dual_platform/weights/best.pt'
shutil.copy(best_pt, 'models/mosquito_yolov8_new.pt')

# 同時備份一份帶日期的版本
date_str = datetime.now().strftime('%Y%m%d')
shutil.copy(best_pt, f'models/mosquito_yolov8_{date_str}.pt')

print("✓ 新模型已保存到 Google Drive:")
print("  - models/mosquito_yolov8_new.pt  ← deploy_model.py 預期路徑")
print(f"  - models/mosquito_yolov8_{date_str}.pt  ← 備份版本")
print("\n⚠️ 兩個平台的部署流程已統一在 deploy_model.py")
print("下一步：請參考下方 Cell 7 (雙平台同步導出)

In [ ]:
import os
from pathlib import Path

# 驗證路徑結構
current_dir = Path.cwd()
models_dir = current_dir / "models"
expected_output = models_dir / "mosquito_yolov8_new.pt"

print("🔍 路徑驗證:")
print(f"  當前工作目錄: {current_dir}")
print(f"  預期輸出路徑: {expected_output}")
print(f"  檔案是否存在: {'✅' if expected_output.exists() else '❌'}")

if expected_output.exists():
    size_mb = expected_output.stat().st_size / 1024 / 1024
    print(f"  檔案大小: {size_mb:.2f} MB")

print("\n🔗 與 deploy_model.py 的串接:")
print(f"  1. Google Drive 同步到本機")
print(f"  2. deploy_model.py 讀取路徑:")
print(f"     config.RELOCATION_BASE_DIR = 'G:\\我的雲端硬碟\\Colab Notebooks\\mosquito-training\\relocated'")
print(f"     training_root = RELOCATION_BASE_DIR.parent  # mosquito-training")
print(f"     src = training_root / 'models' / 'mosquito_yolov8_new.pt'")
print(f"  3. 複製到專案: models/mosquito_yolov8.pt")
print(f"  4. 導出 ONNX: models/mosquito_yolov8.onnx")
print(f"  5. 導出 RKNN: models/mosquito_yolov8.rknn")
print("\n✅ 無斷點設計：所有路徑自動推導，無需手動配置")

### ✅ 路徑驗證（確保與 deploy_model.py 無斷點）

以下檢查確保 Colab 輸出與本機部署腳本的路徑一致：

## 7. (可選) 查看訓練曲線

In [ ]:
from IPython.display import Image, display

# 顯示訓練結果圖表
print("訓練曲線：")
display(Image('runs/detect/mosquito_dual_platform/results.png'))

print("\n混淆矩陣：")
display(Image('runs/detect/mosquito_dual_platform/confusion_matrix.png'))

## 8. 雙平台同步導出 (deploy_model.py)

✅ **統一部署腳本**: `python/deploy_model.py` 已整合兩個平台的導出流程

### 8.1 本地執行 (建議方式)

在本機同步 Google Drive 後執行:

```bash
cd d:/Workspaces/mosquito-pt2d  # 或您的專案路徑
python python/deploy_model.py
```

**自動執行流程**:
1. ✅ 備份舊模型: `models/mosquito_yolov8.pt` → `mosquito_yolov8_backup_<timestamp>.pt`
2. ✅ 複製新模型: `models/mosquito_yolov8_new.pt` → `models/mosquito_yolov8.pt`
3. ✅ 導出 ONNX: `models/mosquito_yolov8.onnx` (640×640, opset 12)
4. ✅ 導出 RKNN: `models/mosquito_yolov8.rknn` (自動生成量化數據集)

### 8.2 進階選項

```bash
# 僅導出 ONNX（跳過 RKNN）
python python/deploy_model.py --skip-rknn

# 使用自訂解析度
python python/deploy_model.py --imgsz 320

# 指定 RKNN 目標平台
python python/deploy_model.py --rknn-target rk3588  # Orange Pi 5
python python/deploy_model.py --rknn-target rk3576  # RDK X5 (如適用)

# 完整選項說明
python python/deploy_model.py --help
```

### 8.3 RDK X5 額外步驟 (hb_mapper)

⚠️ **RDK X5 需要專用工具鏈**: 在 x86 Ubuntu 22.04 機器上執行

```bash
# 1. 安裝 RDK X5 工具鏈
pip install rdkx5-yolo-mapper -i https://mirrors.aliyun.com/pypi/simple/
hb_mapper --version

# 2. 準備校準圖片 (20-50 張)
mkdir calibration_images
# 複製實際場景的蚊子樣本到 calibration_images/

# 3. 下載 RDK 編譯腳本
wget https://raw.githubusercontent.com/D-Robotics/rdk_model_zoo/main/demos/Vision/ultralytics_YOLO/x86/mapper.py

# 4. 編譯為 .bin 模型
python3 mapper.py \
  --onnx models/mosquito_yolov8.onnx \
  --cal-images calibration_images \
  --optimize-level O3 \
  --cal-sample-num 50
```

輸出: `models/mosquito_yolov8_bayese_640x640_nv12_modified.bin`

### 8.4 部署到目標板

**Orange Pi 5** (自動):
- ✅ `models/mosquito_yolov8.rknn` 已可直接部署
- 使用 `rknnlite` 載入推理

**RDK X5** (需手動複製):
- 將 `.bin` 模型複製到板端
- 更新 `python/config.py` 中的模型路徑
- 使用 `hobot_dnn` 載入推理

---

**效能參考**:
| 平台 | 延遲 | 吞吐 | 精度保持 |
|------|------|------|----------|
| RDK X5 | ~7ms | ~247 FPS | >95% |
| Orange Pi 5 | ~15ms | ~60 FPS | >93% |

## 9. deploy_model.py 工作流程圖

```mermaid
graph TD
    A[Colab 訓練完成] --> B[models/mosquito_yolov8_new.pt]
    B --> C[Google Drive 同步到本機]
    C --> D[執行 deploy_model.py]
    
    D --> E[備份舊模型]
    E --> F[複製 new.pt → mosquito_yolov8.pt]
    F --> G{導出 ONNX}
    
    G --> H[mosquito_yolov8.onnx]
    H --> I{平台分支}
    
    I -->|Orange Pi 5| J[自動生成量化數據集]
    J --> K[rknn.build 量化]
    K --> L[mosquito_yolov8.rknn]
    L --> M[部署到 Orange Pi 5]
    
    I -->|RDK X5| N[準備校準圖片]
    N --> O[執行 hb_mapper]
    O --> P[mosquito_yolov8.bin]
    P --> Q[部署到 RDK X5]
```

### 關鍵斷點檢查

✅ **1. Google Drive 同步**
- 確認 `mosquito-training/models/mosquito_yolov8_new.pt` 存在
- 路徑: `<Google Drive>/Colab Notebooks/mosquito-training/models/`

✅ **2. deploy_model.py 執行路徑**
- 必須在專案根目錄執行: `cd d:/Workspaces/mosquito-pt2d`
- 腳本會自動推導 Google Drive 路徑（基於 config.RELOCATION_BASE_DIR）

✅ **3. ONNX 導出**
- 輸出: `models/mosquito_yolov8.onnx`
- 預設 imgsz=640, opset=12 (可用 --imgsz 覆寫)

✅ **4. Orange Pi 5 RKNN 導出**
- 自動從 `sample_collection/confirmed/` 生成量化數據集
- 輸出: `models/mosquito_yolov8.rknn`
- 可用 `--skip-rknn` 跳過

✅ **5. RDK X5 hb_mapper 編譯**
- 需在 x86 環境手動執行（非 deploy_model.py 自動化）
- 輸入: `mosquito_yolov8.onnx`
- 輸出: `mosquito_yolov8_bayese_640x640_nv12_modified.bin`

### 常見問題排查

**Q: 找不到 mosquito_yolov8_new.pt?**
```bash
# 檢查 Google Drive 同步狀態
ls "~/Google Drive/Colab Notebooks/mosquito-training/models/"

# 或手動指定路徑
export RELOCATION_BASE_DIR="/path/to/mosquito-training/relocated"
```

**Q: RKNN 導出失敗?**
- 確認已安裝 `rknn-toolkit2` (需在 x86 機器)
- 使用 `--rknn-no-quant` 嘗試非量化導出

**Q: RDK X5 編譯失敗?**
- 確認使用 Ubuntu 22.04 + Python 3.10
- 校準圖片需與實際場景匹配（光線、角度）
- 參考: https://github.com/D-Robotics/rdk_model_zoo

## 10. (進階) 量化感知訓練 (QAT)

⚠️ **適用於兩個平台**: 如果 PTQ（訓練後量化）精度損失過大（>5%），可使用 QAT 提升精度

### 10.1 為什麼需要 QAT？

兩個平台都使用 INT8 量化推理，可能導致：
- 小目標檢測精度下降
- 色彩空間轉換帶來的損失
- 量化誤差累積

### 10.2 訓練策略

```python
# 在訓練時模擬量化過程
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(
    data='training_dataset/dataset.yaml',
    epochs=150,              # QAT 需要更多 epoch
    imgsz=640,
    batch=16,
    # 量化友好設定
    amp=False,               # 關閉 AMP，使用 FP32 訓練
    optimizer='SGD',         # SGD 對量化更友好
    momentum=0.937,
    weight_decay=0.0005,
    # 資料增強（模擬色彩空間轉換）
    hsv_h=0.015,            # 減少色調增強
    hsv_s=0.4,              # 減少飽和度增強
    hsv_v=0.3,              # 減少亮度增強
)
```

### 10.3 校準圖片建議

| 平台 | 數量 | 要求 |
|------|------|------|
| RDK X5 | 50-200 張 | 640×640，YUV420SP 色彩空間模擬 |
| Orange Pi 5 | 100-300 張 | 640×640，多樣化場景 |

**共同要求**:
- 包含不同光線、角度、背景的蚊子樣本
- 從實際應用環境採集
- 避免過度曝光或欠曝光

### 10.4 精度驗證

**使用 deploy_model.py 自動化測試**:

```bash
# 1. 部署新模型（自動導出 ONNX + RKNN）
python python/deploy_model.py

# 2. Orange Pi 5: 直接測試 .rknn 模型
# (在板端執行推理腳本)

# 3. RDK X5: 編譯 .bin 後測試
python3 mapper.py --onnx models/mosquito_yolov8.onnx --cal-images calibration_images
# (在板端執行推理腳本)
```

**目標精度保持率**: 
- RDK X5: >95% (相對於浮點模型)
- Orange Pi 5: >93% (相對於浮點模型)

### 10.5 與 deploy_model.py 的整合

✅ **完整工作流程**:

1. **Colab 訓練** → `mosquito_yolov8_new.pt`
2. **Google Drive 同步** → 本機
3. **執行 deploy_model.py** → 自動生成:
   - `mosquito_yolov8.pt` (備份舊模型)
   - `mosquito_yolov8.onnx` (兩平台共用)
   - `mosquito_yolov8.rknn` (Orange Pi 5)
4. **手動 RDK X5 編譯** → `mosquito_yolov8.bin`
5. **部署測試** → 驗證精度

✅ **無斷點設計**:
- deploy_model.py 自動讀取 Google Drive 路徑
- 自動生成 RKNN 量化數據集（從 sample_collection）
- 統一模型命名規範（mosquito_yolov8.*）